In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# Sourcing our recorded data, df_f = flappy bird trials, df_e = eeg recordings
df_f1 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_1/FlappyB_1.csv"
df_f2 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_2/FlappyB_2.csv"
df_f3 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_3/FlappyB_3.csv"
df_f4 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_4/FlappyB_4.csv"
df_f5 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_5/FlappyB_5.csv"
df_f6 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_6/FlappyB_6.csv"
df_f7 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_7/FlappyB_7.csv"
df_f8 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_8/FlappyB_8.csv"
df_f9 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_9/FlappyB_9.csv"
df_f10 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_10/FlappyB_10.csv"


df_e1 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_1/FlappyBird_Trial_1_ExG.csv"
df_e2 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_2/FlappyBird_Trial_2_ExG.csv"
df_e3 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_3/FlappyBird_Trial_3_ExG.csv"
df_e4 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_4/FlappyBird_Trial_4_ExG.csv"
df_e5 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_5/FlappyBird_Trial_5_ExG.csv"
df_e6 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_6/FlappyBird_Trial_6_ExG.csv"
df_e7 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_7/FlappyBird_Trial_7_ExG.csv"
df_e8 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_8/FlappyBird_Trial_8_ExG.csv"
df_e9 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_9/FlappyBird_Trial_9_ExG.csv"
df_e10 = "https://raw.githubusercontent.com/MyelinGroup/myelin-fuzzy-logic/main/Our%20Recorded%20Data/FlappyBird_Trial_10/FlappyBird_Trial_10_ExG.csv"


In [4]:
class DataAnalysis():
    def __init__(self, path_EEG, path_FlappyBird, window_size_sec, UNIX, LSL):
        self.df_E = pd.read_csv(path_EEG)
        self.df_F = pd.read_csv(path_FlappyBird)
        self.channels = pd.read_csv(path_EEG).columns[1:]
        self.window = window_size_sec
        self.offset = UNIX - LSL
    
    def find_nearest(self, array, value):
        array = np.asarray(array)
        value_true = array[(np.abs(array - value)).argmin()]
        return value_true
        
    def dataframe_of_events_and_merge(self):
        #This function gets the timestamps of all events Start and Over 
        self.df_E = self.df_E.rename(columns={"TimeStamp": "Timestamp"})
        self.df_E['Timestamp'] = (self.df_E['Timestamp']*1000).astype('int') + self.offset
        df = self.df_E.merge(self.df_F, on='Timestamp', how='outer')
        df = df.sort_values(by="Timestamp").reset_index()
        event_rows = df[(df.Event == 'Start') | (df.Event == 'Over')].index
        return df.iloc[event_rows], df
    
    def get_window(self, timestamp):
        window_df = pd.DataFrame()
        lower = timestamp - self.window * 1000
        upper = timestamp + self.window * 1000
        window_df = self.df_E[(self.df_E.Timestamp >= lower) & (self.df_E.Timestamp <= upper)]
        return window_df
    
    def plot_window(self, event_number):
        events_df, df = self.dataframe_of_events_and_merge()
        timestamp = events_df.iloc[event_number].Timestamp
        window_df = self.get_window(timestamp)
        for channel in self.channels:
            window_array = window_df[channel]
            plt.plot(window_df.Timestamp, window_array, color='red')
            plt.title("Node:" + channel)
            plt.plot(np.ones(1000)*events_df.iloc[event_number]['Timestamp'], np.linspace(min(window_array), max(window_array), 1000))
            plt.grid()
            plt.tight_layout()
            plt.show()
            
    def get_analysis_timestamps(self):
        events_df, df = self.dataframe_of_events_and_merge()
        window_adj = self.window*2
        dictionary = {'neutral':[], 'end':[]}
        events = events_df['Timestamp'].to_numpy()
        # Getting the timestamps of events we need
        for i in range(0, len(events_df), 2):
            time = (events[i+1] - events[i])/1000
            if time > window_adj:
                dictionary['end'].append(events[i+1])
            if time > window_adj*2 and time < 45:
                counter = 1
                while events[i] + window_adj*counter*1000 < events[i+1]:
                    dictionary['neutral'].append(events[i] + window_adj*counter*1000)
                    counter+=1
                    
        #If there's no timestamp corresponding to neutral one exactly, we find the nearest one
        true_array = []
        for element in dictionary['neutral']:
            true_array.append(self.find_nearest(self.df_E['Timestamp'].to_numpy(), element))
        dictionary['neutral'] = true_array
        
        #we replace each element of dictionary with 2-d array of size 9*n
        for i in range(len(dictionary['end'])):
            dictionary['end'][i] = self.get_window(dictionary['end'][i]).to_numpy()
        for j in range(len(dictionary['neutral'])):
            dictionary['neutral'][j] = self.get_window(dictionary['neutral'][j]).to_numpy()
        return dictionary           

In [7]:
model = DataAnalysis(df_e8, df_f8, 2, 1686584209841, 96101166)